# Run OpenQASM Benzene simulation

This sample shows using the QDK tools for visualizing a simulating an OpenQASM program for the Benzene molecule.

First, we import the relevant tools and read the OpenQASM source from a file. We can use the QDK to compile this into a QIR program.

In [ ]:
from qdk import init, TargetProfile
from qdk.openqasm import circuit, compile, run
from qdk.widgets import Circuit, Histogram

from pathlib import Path
src = Path("benzene_diradical_injected_rotation_measurement_circuit_0.qasm").read_text()

init(target_profile=TargetProfile.Base)
qir = compile(src)

Next, we can visualize the program as a circuit using the `Circuit` widget.

In [ ]:
Circuit(circuit(src))

Using the sparse-state simulator, we can run the program without noise and confirm the results visually with the `Histogram` widget.

In [ ]:
# Run using the default sparse simulator
sim_results = run(src, shots=1024)
Histogram(sim_results, labels="kets")

Let's see how this program might run when compiled and scheduled for a neutral atom device. We create a new instance of the `NeutralAtomDevice` class with default parameters and trace the QIR program to see what it looks like when compiled to the native gates and movements of such a device.

In [ ]:
from qdk.simulation import NeutralAtomDevice, NoiseConfig

device = NeutralAtomDevice()
device.show_trace(qir)

Next, we create a noise configuration for the native gates of this neutral atom device, including some probability of qubit loss on movement, and simulate the program targetted for the device using the GPU full-state simulator. You can try varying the noise levels to observe how the results change in the presence of different noise or loss probabilities.

In [ ]:
noise = NoiseConfig()
noise.cz.set_depolarizing(0.05)
noise.sx.set_bitflip(0.01)
noise.mov.loss = 0.001
results = device.simulate(qir, shots=1000, noise=noise, type="gpu")
Histogram(results, labels="kets", items="top-25")

Additionally, you can use the `MoleculeViewer` widget to visualize the structure of the Benzene molecule we've been experimenting with.

In [ ]:
from pathlib import Path
from qdk.widgets import MoleculeViewer

# TODO: This data should come from calling qdk-chemistry Python APIs.
molecule_data = Path("benzene_diradical.structure.xyz").read_text()
cube_data = {
    "alpha_18": Path("MO_alpha_18.cube").read_text(),
    "alpha_19": Path("MO_alpha_19.cube").read_text(),
}

MoleculeViewer(molecule_data=molecule_data, cube_data=cube_data, isoval=0.03)